In [186]:
import pandas as pd
from datetime import datetime
import re

In [187]:
def open_file(filename):
    try:
        f = open(filename, "r")
        text = f.read()
    except:
        f = open(filename, "r",encoding='latin-1')
        text = f.read()
        # encoding='latin-1'
    lines = text.split('\n')
    return lines


def clean_name(name: str) -> str:
    char_add_ons = ['V.O.', "'S COM VOICE",
                    'Cont\'d', 'O.S.', "'S", "VOICE", "INTERCOM"]
    for txt in char_add_ons:
        if txt in name:
            name = name.replace(txt, '')
    return re.sub("[\(\[].*?[\)\]]", "", name.strip())


def get_title(lines):
    series_ind = 0
    for i in range(len(lines)):
        if "STAR TREK: THE NEXT GENERATION" in lines[i] or 'STAR TREK: DEEP SPACE NINE' in lines[i]:
            series_ind = i
            break
    return str(lines[series_ind+2]).strip().replace('"','')


def get_date(lines):
    draft_ind = 0
    for i in range(len(lines)):
        if "FINAL DRAFT" in lines[i]:
            series_ind = i
            break
    try:
        date = str(lines[series_ind+2])
        date = date.strip()
        final = datetime.strptime(date, '%B %d, %Y')
    except:
        print(lines[series_ind+2])
        final = "AHHH"
    return final



In [188]:
def get_quotes(lines):
#    characters = [x for x in lines if '\t\t\t\t\t' in x]
    char_index = [i for i in range(len(lines)) if '\t\t\t\t\t' in lines[i]]
    scenes_index = [j for j in range(len(lines)) if 'INT.' in lines[j] or 'EXT.' in lines[j]]

    data = []

    for i in char_index:
        name = lines[i].replace('\t\t\t\t\t', '')
        if "FADE OUT" in name:
            continue

        j = i+1
        next_line = lines[j]

        quote = ""

        while next_line != '':
            next_line = next_line.replace('\t', ' ')
            quote += re.sub("[\(\[].*?[\)\]]", "", next_line)
            j = j+1
            next_line = lines[j]

        if quote!="":
            print(quote,lines[j],lines[j-1])
            print(j)
            print(scenes_index)
            scene = lines[max([x for x in scenes_index if x < i])]

            data.append([name, quote,scene])

    test_df = pd.DataFrame(data, columns=['character', 'quote','scene'])
    return (test_df)

def clean_location(loc:str):
    views = ['INT.','EXT.']
    output = loc
    for v in views:
        if v in output:
            output = output[output.index(v)+5:]
    if '(' in output:
        output = output[:output.index('(')-1]
    elif '-' in output:
        output = output[:output.index('-')-1]
    return(output)

def get_view(loc:str):
    output = None
    views = ['INT.','EXT.']
    for v in views:
        if v in loc:
            output=v
    return(output)


def create_df(filepath: str) -> pd.DataFrame:
    lines = open_file(filepath)
    test_df = get_quotes(lines)
    test_df['location']=test_df['scene'].apply(clean_location)
    test_df['view']=test_df['scene'].apply(get_view)
    test_df['character'] = test_df['character'].apply(clean_name)
    test_df['episode'] = get_title(lines)
    test_df['date']=get_date(lines)
    return test_df


In [189]:
x=create_df('scripts_tng/156.txt')
x

   The truth is I'm a career man...   you know, married to my job...   Mitzi doesn't understand that...   I like her a lot... but I just   don't think I'm ready for two   marriages if you know what I   mean...  			mean...
132
[159, 191, 257, 290, 303, 454, 468, 634, 818, 820, 964, 1023, 1037, 1120, 1141, 1207, 1243, 1291, 1307, 1312, 1325, 1344, 1352, 1357, 1362, 1373, 1429, 1566, 1765, 1769, 1780, 1787, 1803, 1813, 1893, 1970, 2027, 2093, 2098, 2109, 2195, 2199, 2221, 2366, 2370, 2549, 2637, 2679, 2936, 2967, 3006, 3018]


ValueError: max() arg is an empty sequence

In [191]:
lines = open_file('scripts_ds9/440.txt')
get_title(lines)

'The Maquis, Part I'

In [192]:
full_data=pd.read_csv('complete_data.csv',index_col=0)

# INT. COMMANDER'S OFFICE, COMMANDER'S OFFICE, INT.,

In [193]:
full_data[ full_data.view.isna()] #The Maquis, Part I

,character,quote,scene,location,view,episode,date,series,file
